# Рекомендация тарифов

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

In [5]:
df_train_80, df_test = train_test_split(df,
                                      test_size=0.2,
                                      random_state = 4057)

In [6]:
df_train, df_valid = train_test_split(df_train_80,
                                      test_size=0.25,
                                      random_state = 4057)

In [7]:
print(df_train_80.shape)
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(2571, 5)
(1928, 5)
(643, 5)
(643, 5)


In [8]:
features_train_80 = df_train_80.drop(['is_ultra'], axis=1)
target_train_80 = df_train_80['is_ultra']
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## Исследуйте модели

DecisionTreeClassifier

In [9]:
best_model_dt = None
best_result_dt = 0
best_depth_dt = 0
for i in range(1, 8):
    model_dt = DecisionTreeClassifier(random_state = 4057,
                                      max_depth = i)
    model_dt.fit(features_train, target_train)
    accuracy = model_dt.score(features_valid, target_valid)
    if accuracy > best_result_dt:
        best_result_dt = accuracy
        best_model_dt = model_dt
        best_depth_dt = i
print("Accuracy наилучшей модели на валидационной выборке:", best_result_dt)
print("Глубина дерева наилучшей модели на валидационной выборке:", best_depth_dt)

Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314
Глубина дерева наилучшей модели на валидационной выборке: 7


RandomForestClassifier

In [10]:
best_model_rf = None
best_result_rf = 0
best_depth_rf = 0
best_n_estimators_rf = 0
for i in range(1, 15):
    for k in range(1, 10):
        model_rf = RandomForestClassifier(random_state = 4057, 
                                          n_estimators = i,
                                          max_depth = k)
        model_rf.fit(features_train, target_train)
        accuracy = model_rf.score(features_valid, target_valid)
        if accuracy > best_result_rf:
            best_result_rf = accuracy
            best_model_rf = model_rf
            best_depth_rf = k
            best_n_estimators_rf = i
print("Accuracy наилучшей модели на валидационной выборке:", best_result_rf)
print("Глубина дерева наилучшей модели на валидационной выборке:", best_depth_rf)
print("Количество деревьев наилучшей модели на валидационной выборке:", best_n_estimators_rf)

Accuracy наилучшей модели на валидационной выборке: 0.8133748055987559
Глубина дерева наилучшей модели на валидационной выборке: 8
Количество деревьев наилучшей модели на валидационной выборке: 13


LogisticRegression

In [11]:
model_lr = LogisticRegression(random_state = 4057, 
                              solver='lbfgs',
                              max_iter=1000)
model_lr.fit(features_train, target_train)
accuracy = model_lr.score(features_valid, target_valid)
print("Accuracy модели на валидационной выборке:", accuracy)

Accuracy модели на валидационной выборке: 0.7371695178849145


## Проверьте модель на тестовой выборке

In [12]:
accuracy = best_model_rf.score(features_test, target_test)
print("Accuracy наилучшей модели на тестовой выборке:", accuracy)

Accuracy наилучшей модели на тестовой выборке: 0.8180404354587869


## Вывод

Из всех моделей лучшей оказалась модель случайного леса, но также она дольше всех обучается.